In [1]:
import geopandas as gpd
import cdsapi
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import rioxarray
from rasterio.features import geometry_mask
import plotly.express as px
import glob
import os

In [2]:
IF_DOWNLOAD=False
LOCATION_PATH='locations/chn_admbnda_adm2_ocha_2020.shp'

In [3]:
def query_data(years, dataset, request):

    for year in years:
        request['year']=year
        client = cdsapi.Client()
        client.retrieve(dataset, request, target=f"cloud_cover/era5_cloudcover_{year}.nc")

if IF_DOWNLOAD:
    years=[str(year) for year in range(1990, 2025)]
    dataset = "derived-era5-single-levels-daily-statistics"
    request = {
        "product_type": "reanalysis",
        "variable": ["total_cloud_cover"],
        "month": ["08", "09", "10"],
        "day": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11", "12",
            "13", "14", "15",
            "16", "17", "18",
            "19", "20", "21",
            "22", "23", "24",
            "25", "26", "27",
            "28", "29", "30",
            "31"
        ],
        "daily_statistic": "daily_mean",
        "time_zone": "utc+00:00",
        "frequency": "6_hourly",
        "area": [20.5, 108.5, 18, 111.5]
    }

    query_data(years, dataset, request)
    file_list = sorted(glob.glob("cloud_cover/era5_cloudcover_*.nc"))
    file_list = [f for f in file_list if any(str(y) in f for y in range(1990, 2025))]

    ds_combined = xr.open_mfdataset(
        file_list,
        combine="by_coords",
        parallel=False
    )

    print(ds_combined)
    output_folder = "cloud_cover/processed"
    os.makedirs(output_folder, exist_ok=True)
    ds_combined.to_netcdf(f"{output_folder}/era5_cloudcover_1990_2024_combined.nc")

In [4]:
hainan=gpd.read_file(LOCATION_PATH)
hainan=hainan[hainan['ADM1_PCODE']=='CN046']
hainan.drop(columns=['OBJECTID', 'Admin_type', 'Adm2_CAP', 'ADM2_PCODE', 'ADM1_ZH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_ZH', 'ADM0_PCODE'], inplace=True)

In [ ]:
hainan.plot(edgecolor="black", facecolor="lightblue")
plt.title("Hainan Map")
plt.show()

In [ ]:
date="2024-08-22"

proj=ccrs.PlateCarree()

fig, ax=plt.subplots(figsize=(10,8), subplot_kw={'projection': proj})
hainan = hainan.to_crs("EPSG:4326")
hainan.boundary.plot(ax=ax, edgecolor='red', linewidth=1.5, transform=proj, zorder=2)
ds = xr.open_dataset("cloud_cover/processed/era5_cloudcover_1990_2024_combined.nc")
da_tcc = ds["tcc"]
da_tcc = da_tcc.rio.write_crs("EPSG:4326")
da_day = da_tcc.sel(valid_time=date) 

masked_tcc = da_day.rio.clip(hainan.geometry, hainan.crs, drop=True)

masked_tcc.plot(cmap="viridis", vmin=0, vmax=1)
for _, row in hainan.iterrows():
    city_name=row['ADM2_EN'].split(' ')[0]
    x, y = row.geometry.centroid.x, row.geometry.centroid.y
    ax.text(x,y,city_name, transform=proj, fontsize=8, ha='center', color='white', weight='bold', zorder=3)
plt.title(f"Total Cloud Cover over Hainan ({date})")
plt.show()

In [7]:
cities=['Haikou', 'Sanya']

In [8]:
da = ds["tcc"]
da = da.rio.write_crs("EPSG:4326")

da_september = da.sel(valid_time=da.valid_time.dt.month == 9)

In [ ]:
for city in cities:
    city_shp=hainan[hainan['ADM2_EN']==city]
    city_shp = city_shp.to_crs("EPSG:4326")
    da_clipped = da_september.rio.clip(city_shp.geometry, hainan.crs, drop=True)
    da_avg = da_clipped.mean(dim=["latitude", "longitude"])  

    df = da_avg.to_dataframe(name="cloudcover").reset_index()

    fig = px.line(
        df,
        x="valid_time",
        y="cloudcover",
        markers=True,
        labels={"cloudcover": "Cloudcover", "day": "Day of September"},
        title=f"Daily Total Cloud Cover over Hainan in September in {city}"
    )
    fig.update_layout(hovermode="x unified")
    fig.show()


In [ ]:
for city in cities:
    city_shp=hainan[hainan['ADM2_EN']==city]
    city_shp = city_shp.to_crs("EPSG:4326")
    da_clipped = da_september.rio.clip(city_shp.geometry, hainan.crs, drop=True)
    da_avg = da_clipped.mean(dim=["latitude", "longitude"])  

    df = da_avg.to_dataframe(name="cloudcover").reset_index()
    df.drop(columns=['number', 'spatial_ref'], inplace=True)
    
    df['sunny']=df['cloudcover']<=0.4
    df_sunny=df[df['sunny']==True]
    df_count=df_sunny.groupby(by=df_sunny['valid_time'].dt.year)['sunny'].count()
    df_count = df_count.reset_index()
    df_count.columns = ["valid_time", "sunny"]

    fig = px.bar(df_count, x="valid_time", y="sunny", title=f"the number of sunny days (ttc<40%) in September in {city}")
    fig.show()

